In [20]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from scipy.cluster.hierarchy import linkage, dendrogram, fcluster
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error

In [17]:
df = pd.read_parquet('radar_112/combined_task_all_0807.parquet')
df = df.iloc[:,:6]
df_arr = df.to_numpy()
df_arr.shape

In [18]:
reshaped_arr = df_arr.reshape(132, -1)
reshaped_arr

In [21]:
linked = linkage(reshaped_arr, method='ward')

In [3]:
data = pd.read_parquet("data/parquet_samples/radar_samples_192.168.67.112_412.parquet",
                        engine='fastparquet')
tasks = pd.read_csv('data/task.csv')

In [4]:
divider = DivideData(data, tasks)

In [5]:
participants, num = divider.divide_participants()
col_names = divider.data.columns[:6]
col_names

In [21]:
for i in range(num):
    participant = participants[f'participant_{i + 1}']
    participant = participant.reset_index(drop=True)

    # divide data by task per participant, e.g. participant 1, return the first and the last index of this task
    task_all = divider.divide_task_index(participant)
    
    for j in range(0, 7):
        start, end = task_all[f'T_{j}']
        # prepare an array for storing signal data for 6 channels
        task_arr = np.empty((6, end-start))
        k = 0 # initialize for channels
        for col in col_names:
            # exploded the 2D array to a 1D list, for plotting and computation
            data_series = participant[col]
            data_series_exploded = data_series.explode(col)
            merged_list = data_series_exploded.tolist()
            task_l = merged_list[start:end]
            # normalize
            task_l_mean = np.mean(task_l)
            task_l_std = np.std(task_l)
            task_arr[k,:] = (task_l - task_l_mean) / task_l_std
            # next channel
            k += 1
        file_path = f'participant/part_{i + 1}/task_{j}'
        

In [8]:
df = pd.DataFrame(task_arr.T, columns=col_names)
df.to_parquet(file_path, engine = 'fastparquet')

In [10]:
data_2 = pd.read_parquet(file_path, engine = 'fastparquet')

In [23]:
data_2.shape

In [13]:
data_2_arr = data_2.to_numpy()
data_2_arr = data_2_arr.T
fig, ax = plt.subplots(figsize=(18, 6))
for m in range(6):
    ax.plot(data_2_arr[m], label=col_names[m], alpha = 0.3)
ax.legend()
plt.show()

In [30]:
data_2_mean = data_2.rolling(window=256).mean

In [22]:
data_2_mean.shape

In [17]:
data_2_arr = data_2_mean.to_numpy()
data_2_arr = data_2_arr.T
fig, ax = plt.subplots(figsize=(18, 6))
for m in range(6):
    ax.plot(data_2_arr[m], label=col_names[m], alpha = 0.3)
ax.legend()
plt.show()

In [46]:
data_2_row_mean = data_2.mean(axis=1)


In [47]:
plt.plot(data_2_row_mean, alpha=0.3)
plt.show()

In [42]:
data_2_row_mean.rolling(window=256).mean

In [43]:
import pandas as pd

# Assuming data_2 is your DataFrame
data_2_row_mean = data_2.mean(axis=1)  # Compute mean across each row
data_2_row_mean_rolling = data_2_row_mean.rolling(window=256).mean()


In [45]:
import matplotlib.pyplot as plt

plt.plot(data_2_row_mean_rolling, alpha = 0.3)
plt.xlabel('Index')
plt.ylabel('Rolling Mean')
plt.title('Rolling Mean of Row Means')
plt.show()


In [49]:
data_2_row_mean_rolling_df = pd.DataFrame(data_2_row_mean_rolling, columns=['rolling_mean'])
data_2_row_mean_rolling_df.to_parquet('data_2_row_mean_rolling.parquet')

In [50]:
df = pd.read_parquet('data_2_row_mean_rolling.parquet')
plt.plot(df)